In [22]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.document_loaders import WebBaseLoader


llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o-mini")


class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="Search Wikipedia for requests received as query. Example: Research of Tesla, Inc"
    )

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool to get links to query retrieved from Wikipedia. 
    Example: https://en.wikipedia.org/wiki/Tesla,_Inc.
    """
    args_schema: Type[
        WikipediaSearchToolArgsSchema
    ] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(query)


class DuckDuckGoSearchToolArgsSchema(BaseModel):
    query: str = Field(
    description="Search DuckDuckGo for requests received as query. Example: What is the Covid-19?"
    )

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    Search DuckDuckGo for requests received as query and return the most up-to-date and appropriate link. 
    Example: https://www.mayoclinic.org/diseases-conditions/coronavirus/symptoms-causes/syc-20479963
    """
    args_schema: Type[
        DuckDuckGoSearchToolArgsSchema
    ] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query):
        wrapper = DuckDuckGoSearchAPIWrapper(max_results=2)
        search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="text")
        return search.run(query)

class WebBaseLoaderToolArgsSchema(BaseModel):
    url: str = Field(
        description="Returns content from the URL found in Wikipedia or DuckDuckGo as text."
    )

class WebBaseLoaderTool(BaseTool):
    name = "WebBaseLoaderTool"
    description = """
    Returns contents from the URL found in Wikipedia or DuckDuckGo as text.
    Save the content to a text file.    """
    args_schema: Type[
        WebBaseLoaderToolArgsSchema
    ] = WebBaseLoaderToolArgsSchema

    def _run(self, url):
        wbl = WebBaseLoader(url)
        contents = wbl.load()
        with open('Searching.text', "w", encoding="utf-8") as f:
            page_content = "\n".join(
        [line for doc in contents for line in doc.page_content.splitlines() if line.strip()]
    )
            f.write(page_content)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        WebBaseLoaderTool()
    ],
)

prompt = "Research about the XZ backdoor. After finding url, save the contents of the page as a text file."
result = agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (previously LZM